Function to read f_27 file

In [ ]:
def mean_position_opta_f27(xml_file):
    ''''
    Function that returns a dataframe with all players found in a F27 file, together with
    its mean position based on contacts with ball and tje number of sucessful passes

    Input:
    OPTA F27 xml file


    '''

    import xml.etree.ElementTree as ET
    import pandas as pd

    file = ET.parse(xml_file)

    namelist= []
    xlist =[]
    ylist = []
    poslist = []
    pass_success_list = []

    for node in file.getroot():
        name = node.attrib.get("player_name")
        x = node.attrib.get("x")
        y = node.attrib.get("y")
        pos = node.attrib.get("position")
        pass_success = node.attrib.get("pass_success")
        namelist.append(name)
        xlist.append(x)
        ylist.append(y)
        poslist.append(pos)
        pass_success_list.append(pass_success)


    
    df=pd.DataFrame(data = list(zip(namelist,xlist,ylist,poslist, pass_success_list)),
                    columns = ["Passer","x","y","Position_Passer","PassSuccess"])
    
    return df.sort_values(by=["Position_Passer",'Passer']).reset_index(drop=True)

In [16]:
test=mean_position_opta_f27('C:/Users/Felix/Desktop/Máster  BD Deporte/Módulo 7 - Proveedores de Datos Deportivos/f27/pass_matrix_23_2019_g1074821_t957.xml')
test

,Passer,x,y,Position_Passer,PassSuccess
0,Dimitrios Siovas,39.64,75.41,Defender,31
1,Rodrigo Tarín,33.80,44.08,Defender,36
2,Unai Bustinza,35.79,23.62,Defender,26
3,Martin Braithwaite,66.27,44.32,Forward,9
4,Youssef En-Nesyri,68.63,56.16,Forward,9
5,Óscar Rodríguez,64.74,68.66,Forward,19
6,Iván Cuéllar,12.40,53.23,Goalkeeper,21
7,Javier Eraso,49.60,27.68,Midfielder,15
8,Jonathan Silva,56.95,90.17,Midfielder,16
9,Roberto Rosales,55.98,9.18,Midfielder,15


In [33]:
def pass_matrix(xml_file, pivot_table=False, color=False):
    ''''
    Function that returns a dataframe with all players found in a F27 file, together with
    its mean position based on contacts with ball and tje number of sucessful passes

    Input:
    OPTA F27 xml file


    '''

    import xml.etree.ElementTree as ET
    import pandas as pd

    file = ET.parse(xml_file)

    passer = []
    receiver = []
    passeslist = []

    for node in file.getroot():
        for players in node:
            passes = players.text
            name = players.attrib.get("player_name")
            passer.append(node.attrib.get("player_name"))
            receiver.append(name)
            passeslist.append(int(passes))


    df=pd.DataFrame(data = list(zip(passer,receiver,passeslist)),
                    columns = ["Passer","Receiver","Number of Succesful Passes"])
    
    if pivot_table==True:
    # Crear la tabla de frecuencias
        df = df.pivot_table(values='Number of Succesful Passes', index='Passer', columns='Receiver', fill_value=0)
    
    if color==True:
        return df.style.background_gradient( axis=None)
    else:
        return df

In [37]:
test2=pass_matrix('C:/Users/Felix/Desktop/Máster  BD Deporte/Módulo 7 - Proveedores de Datos Deportivos/f27/pass_matrix_23_2019_g1074821_t957.xml',pivot_table=True)
test2

Receiver,Dimitrios Siovas,Iván Cuéllar,Javier Eraso,Jonathan Silva,José Arnáiz,Marc Navarro,Martin Braithwaite,Recio,Roberto Rosales,Rodrigo Tarín,Rubén Pérez,Unai Bustinza,Youssef En-Nesyri,Óscar Rodríguez
Passer,,,,,,,,,,,,,,
Dimitrios Siovas,0.0,3.0,1.0,6.0,0.0,1.0,2.0,2.0,0.0,6.0,3.0,1.0,2.0,4.0
Iván Cuéllar,10.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0
Javier Eraso,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,1.0,2.0,3.0,0.0,0.0
Jonathan Silva,4.0,1.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,2.0
José Arnáiz,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Martin Braithwaite,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,3.0
Recio,0.0,0.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Roberto Rosales,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,2.0,4.0,3.0,0.0
Rodrigo Tarín,10.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,12.0,2.0,1.0


In [ ]:
def Plotter(df):
    import matplotlib.pyplot as plt
    from matplotlib.patches import Arc

    for i in range(df.count()):
        plt.arrow(x = df.iloc[i,1],
                    y = df.iloc[i,2],
                    dx = (df.iloc[i+1,1]-df.iloc[i,1]),
                    dy = (ycord[1]-ycord[0]),
                    linewidth = int(number),
                    alpha = (int(number)/num_max_pases),
                    length_includes_head=True,
                    color = "red")